# Query twitter API

In [12]:
import os
import requests
# from dotenv import load_dotenv, find_dotenv
from datetime import datetime
import pandas as pd
from dotenv import load_dotenv, find_dotenv


search_url = "https://api.twitter.com/2/tweets/search/recent"
# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields

def get_token():
    env_path = os.path.join(os.path.dirname(os.getcwd()),'.env')
    env_path = find_dotenv() # automatic find 
    load_dotenv(env_path)
    token = os.getenv('TWITTER_BEARER_TOKEN','Key missing in env settings')
    return token

def last_date_db():
    """
    Method to retrieve the last date in the DB
    """
    # Construct a BigQuery client object.

    query = """
        SELECT created_at
        FROM `wagon-bootcamp-802.my_dataset.new_table`
        ORDER BY created_at DESC
        LIMIT 1
    """
    df = pd.read_gbq(query, dialect='standard')
    return df.iloc[0]['created_at'].tz_localize(None).isoformat()+'Z'


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """
    bearer_token = get_token()
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


def query_twitter(start_time=None, next_token=None):
    query_params = {'query': '#F1',
                    'tweet.fields':'created_at,lang',
                    'start_time':start_time,
                    'max_results' : '100',
                    'next_token': next_token
                   }
    json_response = connect_to_endpoint(search_url, query_params)
    return json_response


def fetching_tweets(response):
    if response['meta'].get('next_token',False):
        df = pd.DataFrame(response['data'])[['text', 'created_at', 'id', 'lang']]
    else:
        df = pd.DataFrame(response['data'])[['text', 'created_at', 'id', 'lang']].iloc[:-1]
    df = df[df['lang'] == 'en']
    df['created_at'] = pd.to_datetime(df['created_at'])
    return df

def main():
    '''
    Pushing results to GBQ
    '''
    most_recent_dt = last_date_db()
    response = query_twitter(most_recent_dt)
    data = pd.DataFrame()
    while response['meta'].get('next_token',False):
        data = data.append(fetching_tweets(response), ignore_index=True)
        response = query_twitter(most_recent_dt,response['meta'].get('next_token',False))
    data = data.append(fetching_tweets(response), ignore_index=True)
#     table_id = 'wagon-bootcamp-802.my_dataset.twitter_table'
#     data.to_gbq(table_id, if_exists='append')
    return data

In [23]:
fetching_tweets({'data': [{
                'lang': 'en',
                'created_at': '2022-07-06T13:53:07.000Z',
                'id': '1544680846883528706',
                'text': '“This weekend won’t be smooth sailing” says Championship Leader Max Verstappen #Formula1 #F1 #F1News #Motorsport https://t.co/ncnzaoqvcd'
                    }],
            'meta': {'newest_id': '1544680846883528706',
                'oldest_id': '1544679599849623552',
                'result_count': 100
                    }
            })

,text,created_at,id,lang


In [20]:
df_test_comparison = pd.DataFrame({
                'la': '“This weekend won’t be smooth sailing” says Championship Leader Max Verstappen #Formula1 #F1 #F1News #Motorsport https://t.co/ncnzaoqvcd',
                'created_at': '2022-07-06T13:53:07.000Z',
                'id': '1544680846883528706',
                'lang': 'en'
                    },index = [0])
df_test_comparison['created_at'] = pd.to_datetime(df_test_comparison['created_at'])
try:
    pd.testing.assert_frame_equal(df_test_from_json,df_test_comparison)
except:
    assert False
assert True

AssertionError: 

In [24]:
pd.DataFrame({
                'text': [],
                'created_at': [],
                'id': [],
                'lang': [] 
                    },index = [])

,text,created_at,id,lang
